In [ ]:
#!pip uninstall matplotlib
!pip install parse
!pip install matplotlib --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=7fe2c5ae340f3d438cc31fef709b7409286ee6d69e9aeeabd8faee3c80ca2f82
  Stored in directory: /root/.cache/pip/wheels/9c/aa/cc/f2228050ccb40f22144b073f15a2c84f11204f29fc0dce028e
Successfully built parse
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.2 MB 5.4 MB/s 
     |████████████████████████████████| 944 kB 53.6 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you h

In [ ]:
!pip show matplotlib

Name: matplotlib
Version: 3.5.2
Summary: Python plotting package
Home-page: https://matplotlib.org
Author: John D. Hunter, Michael Droettboom
Author-email: matplotlib-users@python.org
License: PSF
Location: /usr/local/lib/python3.7/dist-packages
Requires: pillow, kiwisolver, pyparsing, python-dateutil, packaging, fonttools, cycler, numpy
Required-by: yellowbrick, seaborn, scikit-image, pycocotools, plotnine, pandas-profiling, mlxtend, mizani, missingno, matplotlib-venn, keras-vis, imgaug, fbprophet, fastai, descartes, daft, arviz


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
colab_datadir = '/content/drive/MyDrive/sharedCodeBase/FamilyAnalysis/'
local_datadir = './'
datadir = colab_datadir
import sys
sys.path.append(datadir)

In [ ]:
import pandas as pd
import numpy as np

from parse import *
import os
import sklearn
from sklearn.datasets import make_blobs
from matplotlib import pyplot as plt
#%matplotlib auto
import matplotlib.image as mpimg 
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.collections import PolyCollection
from collections import defaultdict
import datetime
from datetime import timedelta
import argparse
from myHelper import *
from graphHelper import *

In [ ]:
'''
cmd不方便调试，就建了一个类充当参数输入的接口
'''

class Myclass():
    MinCountOfPoints=6 #这里我没改，因为分长时间和短时间计算时，我手动一起调了（详见最后一个part）
    MinCountOfPointsForAll=-1
    SizeOfGrid='1' # 不需要调 
    figure=datadir+'floorplan01.png' # 默认
    path=datadir+'results_both/' # 默认
    res=datadir+'figures_both/' # 默认
    bySecond=False
    IntervalOfMinutesOrSeconds=1 # 1分钟 1个点  
    setConfig=True #False

arguments=Myclass()


In [ ]:

##########################################
# Choose Family
##########################################
family_config = chooseFamily()

max_x = family_config.size[1]
min_x = family_config.size[0]
max_y = family_config.size[3]
min_y = family_config.size[2]

if family_config.name == "ParentsOfSun":
  arguments.figure = datadir+'floorplan01.png'
  arguments.path = datadir+'family1_results_both/'
  arguments.res=datadir+'family1_figures_both/'
elif family_config.name == "UncleOfSun":
  arguments.figure = datadir+'floorplan02.png'
  arguments.path = datadir+'family2_results_both/'
  arguments.res=datadir+'family2_figures_both/'
elif family_config.name == "AuntOfSun":
  arguments.figure = datadir+'floorplan03.png'
  arguments.path = datadir+'family3_results_tag0/'
  arguments.res=datadir+'family3_figures_tag0/'
else:
  print("Family doesn't exist !\n")
  exit(0)

unit = float(arguments.SizeOfGrid)  #网格单元长宽m
column_num = int((max_x-min_x)//unit)
row_num = int((max_y-min_y)//unit)
map_img = mpimg.imread(arguments.figure)

# time epsilon to select the data point
epsilon = 2 #timedelta(minutes=10)
min_count = arguments.MinCountOfPoints

# define time
year = 2022
input_path = arguments.path
res_path = arguments.res
C = 0

In [ ]:
"""
Read Data
"""
n = arguments.IntervalOfMinutesOrSeconds
if n == 1:
    if arguments.bySecond:
        end_suffix = '_dataBySecond.xlsx'
    else:
        end_suffix = '_dataByMinute.xlsx'
else:
    if arguments.bySecond:
        end_suffix = '_dataBy'+str(n)+'Seconds.xlsx'
    else:
        end_suffix = '_dataBy'+str(n)+'Minutes.xlsx'
    

# read files
files = [f for f in os.listdir(input_path) if f.endswith(end_suffix)]
# read data
print("Files read are :", files)
tags_order = []
all_points_array = []
dict_df = {}
list_df = []
# data collection for one
data_collection = []

if arguments.MinCountOfPointsForAll == -1:
    C = min_count * len(files)
else:
    C = arguments.MinCountOfPointsForAll

    

for f in files:
    df = pd.read_excel(open(input_path+f,'rb')) 

    # 修改门口坐标
    print(df)
    
    # 修改门口坐标-tag0
    # print(df)
    
# tag0
#     df.loc[(((abs(df['x值']-2.1175)<0.0001) &(abs(df['y值']-6.27)<0.0001))|
#             ((abs(df['x值']-1.14875)<0.0001) &(abs(df['y值']-1.75875)<0.0001))|
#             ((abs(df['x值']-1.4475845)<0.0001) &(abs(df['y值']-6.7912643)<0.0001))|
#             ((abs(df['x值']-1.44)<0.0001) &(abs(df['y值']-6.77999999999999)<0.0001))|
#             ((abs(df['x值']-1.54444444444445)<0.0001) &(abs(df['y值']-6.5888888888889)<0.0001))|
#             ((abs(df['x值']-1.53142857142857)<0.0001) &(abs(df['y值']-6.65428571428572)<0.0001))|
#             ((abs(df['x值']- -0.04)<0.0001) &(abs(df['y值']-6.86000000000001)<0.0001))|
#             ((abs(df['x值']-1.44666666666667)<0.0001) &(abs(df['y值']-6.57833333333332)<0.0001))|
#             ((abs(df['x值']-1.364)<0.0001) &(abs(df['y值']-5.886)<0.0001))|
#             ((abs(df['x值']-2.05)<0.0001) &(abs(df['y值']-7.88333333333333)<0.0001))|
#             ((abs(df['x值']-1.81)<0.0001) &(abs(df['y值']-7.94)<0.0001))|
#             ((abs(df['x值']- -0.025)<0.0001) &(abs(df['y值']-5.41833333333333)<0.0001))|
#             ((abs(df['x值']- -0.04)<0.0001) &(abs(df['y值']-7.05500000000001)<0.0001))|
#             ((abs(df['x值']-0.38)<0.0001) &(abs(df['y值']-7.89999999999999)<0.0001))|
#             ((abs(df['x值']-0.35)<0.0001) &(abs(df['y值']-7.505)<0.0001))|
#             ((abs(df['x值']-1.11)<0.0001) &(abs(df['y值']-6.66000000000001)<0.0001))|
#             ((abs(df['x值']-1.40833333333333)<0.0001) &(abs(df['y值']-6.71499999999999)<0.0001))|
#             ((abs(df['x值']-1.34714285714286)<0.0001) &(abs(df['y值']-6.56857142857143)<0.0001))|
#             ((abs(df['x值']-1.56111111111111)<0.0001) &(abs(df['y值']-6.63333333333333)<0.0001))|
#             ((abs(df['x值']-1.18)<0.0001) &(abs(df['y值']-6.86200000000001)<0.0001))|
#             ((abs(df['x值']-0.56125)<0.0001) &(abs(df['y值']-3.25749999999999)<0.0001))|
#             ((abs(df['x值']-1.5325)<0.0001) &(abs(df['y值']-6.5675)<0.0001))|
#             ((abs(df['x值']-0.817777777777778)<0.0001) &(abs(df['y值']-6.74333333333334)<0.0001))|
#             ((abs(df['x值']-1.215)<0.0001) &(abs(df['y值']-6.83)<0.0001))|
#             ((abs(df['x值']-1.343)<0.0001) &(abs(df['y值']-6.68599999999999)<0.0001))|
#             ((abs(df['x值']-1.35333333333333)<0.0001) &(abs(df['y值']-6.66166666666667)<0.0001))|
#             ((abs(df['x值']-0.954444444444446)<0.0001) &(abs(df['y值']-6.77666666666666)<0.0001))|
#             ((abs(df['x值']-1.28)<0.0001) &(abs(df['y值']-6.62)<0.0001))|
#             ((abs(df['x值']-1.316)<0.0001) &(abs(df['y值']-6.656)<0.0001))|
#             ((abs(df['x值']-1.16)<0.0001) &(abs(df['y值']-2.135)<0.0001))|
#             ((abs(df['x值']-1.21142857142857)<0.0001) &(abs(df['y值']-6.85142857142858)<0.0001))|
#             ((abs(df['x值']-1.418)<0.0001) &(abs(df['y值']-6.916)<0.0001))|
#             ((abs(df['x值']-1.2475)<0.0001) &(abs(df['y值']-1.9625)<0.0001))|
#             ((abs(df['x值']-1.58)<0.0001) &(abs(df['y值']-6.62)<0.0001))|
#             ((abs(df['x值']-1.552)<0.0001) &(abs(df['y值']-6.85199999999999)<0.0001))|
#             ((abs(df['x值']-1.47)<0.0001) &(abs(df['y值']-6.35)<0.0001))|
#             ((abs(df['x值']-1.146)<0.0001) &(abs(df['y值']-6.39600000000001)<0.0001))|
#             ((abs(df['x值']-1.31)<0.0001) &(abs(df['y值']-6.43666666666667)<0.0001))|
#             ((abs(df['x值']-1.57444444444444)<0.0001) &(abs(df['y值']-6.56888888888888)<0.0001))|
#             ((abs(df['x值']-1.2925)<0.0001) &(abs(df['y值']-6.75999999999999)<0.0001))|
#             ((abs(df['x值']-1.259)<0.0001) &(abs(df['y值']-6.73599999999999)<0.0001))|
#             ((abs(df['x值']-1.27666666666667)<0.0001) &(abs(df['y值']-6.74833333333333)<0.0001))|
#             ((abs(df['x值']-1.255)<0.0001) &(abs(df['y值']-6.63)<0.0001))|
#             ((abs(df['x值']-1.08333333333333)<0.0001) &(abs(df['y值']-6.53333333333334)<0.0001))
#             ), 'x值'] = 3.0
           
#     df.loc[((abs(df['x值'] - 3.0)<0.0001) & 
#             ((abs(df['y值']-6.27)<0.0001)| 
#              (abs(df['y值']-1.75875)<0.0001)|
#              (abs(df['y值']-6.7912643)<0.0001)|
#              (abs(df['y值']-6.77999999999999)<0.0001)|
#              (abs(df['y值']-6.5888888888889)<0.0001)|
#              (abs(df['y值']-6.77999999999999)<0.0001)|
#              (abs(df['y值']-6.5888888888889)<0.0001)|
#              (abs(df['y值']-6.65428571428572)<0.0001)|
#              (abs(df['y值']-6.86000000000001)<0.0001)|
#              (abs(df['y值']-6.57833333333332)<0.0001)|
#              (abs(df['y值']-5.886)<0.0001)|
#              (abs(df['y值']-7.88333333333333)<0.0001)|
#              (abs(df['y值']-7.94)<0.0001)|
#              (abs(df['y值']-5.41833333333333)<0.0001)|
#              (abs(df['y值']-7.05500000000001)<0.0001)|
#              (abs(df['y值']-7.89999999999999)<0.0001)|
#              (abs(df['y值']-7.505)<0.0001)|
#              (abs(df['y值']-6.66000000000001)<0.0001)|
#              (abs(df['y值']-6.71499999999999)<0.0001)|
#              (abs(df['y值']-6.56857142857143)<0.0001)|
#              (abs(df['y值']-6.63333333333333)<0.0001)|
#              (abs(df['y值']-6.86200000000001)<0.0001)|
#              (abs(df['y值']-3.25749999999999)<0.0001)|
#              (abs(df['y值']-6.5675)<0.0001)|
#              (abs(df['y值']-6.83)<0.0001)|
#              (abs(df['y值']-6.74333333333334)<0.0001)|
#              (abs(df['y值']-6.68599999999999)<0.0001)|
#              (abs(df['y值']-6.66166666666667)<0.0001)|
#              (abs(df['y值']-6.77666666666666)<0.0001)|
#              (abs(df['y值']-6.62)<0.0001)|
#              (abs(df['y值']-6.656)<0.0001)|
#              (abs(df['y值']-2.135)<0.0001)|
#              (abs(df['y值']-6.85142857142858)<0.0001)|
#              (abs(df['y值']-6.916)<0.0001)|
#              (abs(df['y值']-1.9625)<0.0001)|
#              (abs(df['y值']-6.62)<0.0001)|
#              (abs(df['y值']-6.85199999999999)<0.0001)|
#              (abs(df['y值']-6.35)<0.0001)|
#              (abs(df['y值']-6.39600000000001)<0.0001)|
#              (abs(df['y值']-6.43666666666667)<0.0001)|
#              (abs(df['y值']-6.56888888888888)<0.0001)|
#              (abs(df['y值']-6.75999999999999)<0.0001)|
#              (abs(df['y值']-6.73599999999999)<0.0001)|
#              (abs(df['y值']-6.74833333333333)<0.0001)|
#              (abs(df['y值']-6.63)<0.0001)|
#              (abs(df['y值']-6.53333333333334)<0.0001)
#              )),'y值'] = 5.0
    
# # tag1
#     df.loc[(((abs(df['x值']- 0.3225)<0.0001)& (abs(df['y值']-6.3725)<0.0001))|
#             ((abs(df['x值']- -0.14)<0.0001)& (abs(df['y值']-5.12)<0.0001))|
#             ((abs(df['x值']- 1.61333333333333)<0.0001) & (abs(df['y值']-6.97166666666668)<0.0001))|
#             ((abs(df['x值']- 0.378333333333334)<0.0001)& (abs(df['y值']-6.40166666666666)<0.0001))|
#             ((abs(df['x值']- 0.656999999999999)<0.0001) & (abs(df['y值']-6.48)<0.0001))|
#             ((abs(df['x值']- 0.55)<0.0001) & (abs(df['y值']-6.47000000000001)<0.0001))|
#             ((abs(df['x值']- 2.77142857142857)<0.0001) & (abs(df['y值']-4.45285714285714)<0.0001))|
#             ((abs(df['x值']- -0.03)<0.0001) & (abs(df['y值']-6.97000000000001)<0.0001))|
#             ((abs(df['x值']- 1.08333333333333)<0.0001)& (abs(df['y值']-7.36166666666668)<0.0001))|
#             ((abs(df['x值']- -0.065)<0.0001) & (abs(df['y值']-7.455)<0.0001))|
#             ((abs(df['x值']- 0.903999999999998)<0.0001) & (abs(df['y值']-6.63)<0.0001))|
#             ((abs(df['x值']- 0.725000000000001)<0.0001) & (abs(df['y值']-6.69999999999999)<0.0001))|
#             ((abs(df['x值']- 0.855714285714285)<0.0001) & (abs(df['y值']-7.46857142857142)<0.0001))|
#             ((abs(df['x值']- 0.76)<0.0001) & (abs(df['y值']-6.76666666666666)<0.0001))|
#             ((abs(df['x值']- 0.335000000000001)<0.0001) & (abs(df['y值']-5.75)<0.0001))|
#             ((abs(df['x值']- -0.205)<0.0001) & (abs(df['y值']-6.36000000000001)<0.0001))|
#             ((abs(df['x值']- 1.75)<0.0001) & (abs(df['y值']-9.435)<0.0001))|
#             ((abs(df['x值']- 0.86)<0.0001) & (abs(df['y值']-6.58)<0.0001))|
#             ((abs(df['x值']- 0.414999999999999)<0.0001) & (abs(df['y值']-6.81)<0.0001))|
#             ((abs(df['x值']- 2.8375)<0.0001) & (abs(df['y值']-4.585)<0.0001))|
#             ((abs(df['x值']- 0.445)<0.0001) & (abs(df['y值']-6.50999999999999)<0.0001))|
#             ((abs(df['x值']- 0.899999999999999)<0.0001) & (abs(df['y值']-5.31999999999999)<0.0001))|
#             ((abs(df['x值']- 0.04)<0.0001) & (abs(df['y值']-6.13)<0.0001))|
#             ((abs(df['x值']- 1.05)<0.0001) & (abs(df['y值']-6.65333333333333)<0.0001))|
#             ((abs(df['x值']- 0.82)<0.0001) & (abs(df['y值']-7.19428571428572)<0.0001))|
#             ((abs(df['x值']- 1.11)<0.0001) & (abs(df['y值']-6.60000000000001)<0.0001))|
#             ((abs(df['x值']- 0.355)<0.0001) & (abs(df['y值']-6.60833333333333)<0.0001))|
#             ((abs(df['x值']- 0.420000000000001)<0.0001) & (abs(df['y值']-7.06999999999999)<0.0001))|
#             ((abs(df['x值']- 0.44)<0.0001) & (abs(df['y值']-5.56999999999999)<0.0001))|
#             ((abs(df['x值']- 2.775)<0.0001) & (abs(df['y值']-4.63)<0.0001))|
#             ((abs(df['x值']- 3.88)<0.0001) & (abs(df['y值']-8.41750000000001)<0.0001))|
#             ((abs(df['x值']- 1.393)<0.0001) & (abs(df['y值']-7.408)<0.0001))|
#             ((abs(df['x值']- 0.292)<0.0001) & (abs(df['y值']-6.08600000000001)<0.0001))      
#             ), 'x值'] = 3.0
           
#     df.loc[((abs(df['x值'] - 3.0)<0.0001) & 
#             ((abs(df['y值']-6.3725)<0.0001) | 
#              (abs(df['y值']-5.12)<0.0001)|
#              (abs(df['y值']-6.97166666666668)<0.0001)|
#              (abs(df['y值']-6.40166666666666)<0.0001)|
#              (abs(df['y值']-6.48)<0.0001)|
#              (abs(df['y值']-6.47000000000001)<0.0001)|
#              (abs(df['y值']-4.45285714285714)<0.0001)|
#              (abs(df['y值']-6.97000000000001)<0.0001)|
#              (abs(df['y值']-7.36166666666668)<0.0001)|
#              (abs(df['y值']-7.455)<0.0001)|
#              (abs(df['y值']-6.63)<0.0001)|
#              (abs(df['y值']-6.69999999999999)<0.0001)|
#              (abs(df['y值']-7.46857142857142)<0.0001)|             
#              (abs(df['y值']-6.76666666666666)<0.0001)|
#              (abs(df['y值']-5.75)<0.0001)|
#              (abs(df['y值']-6.36000000000001)<0.0001)|
#              (abs(df['y值']-9.435)<0.0001)|
#              (abs(df['y值']-6.58)<0.0001)|
#              (abs(df['y值']-6.81)<0.0001)|
#              (abs(df['y值']-4.585)<0.0001)|
#              (abs(df['y值']-6.50999999999999)<0.0001)|
#              (abs(df['y值']-5.31999999999999)<0.0001)|       
#              (abs(df['y值']-6.13)<0.0001)|
#              (abs(df['y值']-6.65333333333333)<0.0001)|
#              (abs(df['y值']-7.19428571428572)<0.0001)|      
#              (abs(df['y值']-6.60000000000001)<0.0001)|
#              (abs(df['y值']-6.60833333333333)<0.0001)|      
#              (abs(df['y值']-7.06999999999999)<0.0001)|
#              (abs(df['y值']-5.56999999999999)<0.0001)|      
#              (abs(df['y值']-4.63)<0.0001)|
#              (abs(df['y值']-8.41750000000001)<0.0001)|
#              (abs(df['y值']-7.408)<0.0001)|
#              (abs(df['y值']-6.08600000000001)<0.0001)
#              )),'y值'] = 5.0

    
    ori_df = df.copy()# store the non-shifted points
    # shift data
    if arguments.setConfig :
        print("==========>Start Shifting<============")
        df = shift(family_config,df)
        print("==========>End Shifting<============")
        # print(df_temp)

    # one copy of df
    df_copy = df.copy()

    parsed = parse("{}_{}_{}",f)
    my_date = str(year)+'-'+parsed[1][:2]+'-'+parsed[1][2:]
    date = str(dt.datetime.strptime(my_date, '%Y-%m-%d').date())

    points = []
    old_points = []
    key = parsed[0]+"_"+my_date 

    for i in range(len(df['hour'])):
        point = [df['x值'][i],df['y值'][i]]
        old_point = [ori_df['x值'][i],ori_df['y值'][i]]
        points.append(point)
        old_points.append(old_point)
        data_collection.append(point)
            
        # define new time for all in one
        # TODO : if necessary, to change all dataframe time
        new_time = pd.to_datetime(date + " " + df['time'][i].strftime("%H:%M:%S"))
        #df_copy['time'][i] = new_time
        df_copy.at[i,'time'] = new_time

    points_array = np.array(points)
    old_points_array = np.array(old_points)
    
    # create non-shifted scatter
    createScatter(old_points_array,key,map_img,res_path,max_x,min_x,max_y,min_y)
    #createScatter(points_array,key,map_img,res_path,max_x,min_x,max_y,min_y)

    tags_order.append(parsed[0])
    all_points_array.append(points_array)
        
    dict_df[key] =df_copy # change the time
    #print(all_points_array)
    list_df.append(df_copy)
    
print("==========>Finish Reading<============")
'''
dict_df:
key: 'tag0_2022_2_19'等
data: 当天的时间-点值 dataframe
'''

In [ ]:
# collect all data by concatinate all dataframe to one
data_collection_array = np.array(data_collection)
df_week = pd.concat(list_df,sort=False)
# NOTE : necessary ! to remove the index for series in dataframe
df_week.reset_index(drop=True, inplace=True) 
print("read files :",files)
print(len(all_points_array))

nb_record = len(all_points_array)

# For sub plots(shifted)
# closed for now
"""
fig = plt.figure(figsize = (12,nb_record//4*3)) # NOTE: 可以修改出图尺寸
# create scatter
for i in range(0,nb_record):
    ax = plt.subplot(math.ceil(nb_record/4),4,i+1)
    # NOTE: 3 rows, and 8 columns
    #ax.text(0.5, 0.5, str((3,8,i)), fontsize=18, ha='center')
    #plt.title(my_date)
    ax.scatter(all_points_array[i][:,0],all_points_array[i][:,1],s=20)
"""
# NOTE : necessary !
createPath(res_path)
#fig.savefig(res_path+"shifted_scatter.png",dpi=300)
#plt.close(fig)

# For all in one plot
# fig = plt.figure(figsize = (15, 15)) # NOTE: 可以修改出图尺寸
# create scatter


In [ ]:
'''
shifted_scatter_allIn 数据清理后，总共14天的散点图
'''
fig,ax = plt.subplots(figsize=(int(max_x-min_x),int(max_y-min_y)))
ax.scatter(data_collection_array[:,0],data_collection_array[:,1],s=20) # data_collection_array: 第一列：x;第二列：y 排数=点数
plt.imshow(map_img,zorder = 0, extent=[min_x,max_x,min_y,max_y])
fig.savefig(res_path+"shifted_scatter_allInOne.png",dpi=300)
plt.close(fig)

In [ ]:
number_cluster={}  # key=日期 value=当天的聚类数
totaltimes=pd.DataFrame(columns=['date','TimeAtHome']) # 每
dict_zones_dict={}

In [ ]:
'''
统计每天的在家时间
'''
for key in dict_df.keys():
    data = collectData(dict_df,key)
    groups,grids = createGrid(data,key,column_num,row_num,max_x,min_x,max_y,min_y)
    data=WashData(data)
    # 记录每一天的在家时间
    totaltime=len(data)
    new_time=pd.DataFrame([[key, totaltime]],columns=['date','TimeAtHome'])
    totaltimes=totaltimes.append(new_time,ignore_index=True)

print(totaltimes)


    

In [ ]:
'''
整个14天的分区表示——聚类+画成热力图
'''
key="All"
data_allInOne = collectDataAllInOne(df_week)
groups,grids = createGrid(data_allInOne,key,column_num,row_num,max_x,min_x,max_y,min_y)
createHeatmap(grids,"heatmap_allInOne",map_img,res_path,unit,max_x,min_x,max_y,min_y)

# create excel writer object 输出每个点对应的网格
writer = pd.ExcelWriter(datadir+'output.xlsx')
# write dataframe to excel
data_allInOne.to_excel(writer)
# save the excel
writer.save()
    
 # 删去点数小于min_count的网格中的点——剩余的点更加集中，便于聚类
choice_data_all=choose_spots(data_allInOne,groups,60)
x=choice_data_all['x']
y=choice_data_all['y']
# centers: 聚类后每一类的中心点坐标——根据中心点划定zone的范围
centers_all,y_pre_all = K_m(x,y,key,number_cluster,map_img,res_path,max_x,min_x,max_y,min_y)
choice_data_all['zone']=y_pre_all


dict_zones_all= get_dict_zones(choice_data_all,number_cluster[key])
zones_info_forall=get_zone_info("All",dict_zones_all,res_path)
intervals_in_zones,details = getTimeIntervalsForZones(dict_zones_all,data_allInOne,epsilon)

new_grids = createHeatmapByTime(dict_zones_all,"heatmap_allInOne",map_img,res_path,row_num,column_num,unit,max_x,min_x,max_y,min_y,groups)
createHeatmapByTime2(new_grids,"heatmap_allInOne",map_img,res_path,unit,max_x,min_x,max_y,min_y)
createTimeLine(key,details,res_path)


In [ ]:


'''
按在家时间长短分为两类分析
首先处理数据，将数据分为两类并整合
'''
    
list_long_df=[]
list_short_df=[]
list_mid_df=[]

for idx,row in totaltimes.iterrows():
    if row['TimeAtHome']>600: # 10h以上 长时间
        list_long_df.append(collectData(dict_df,row['date']))
    elif row['TimeAtHome']<300: # 5h以下 短时间
        list_short_df.append(collectData(dict_df,row['date']))
    else:
        list_mid_df.append(collectData(dict_df,row['date']))

df_long = pd.concat(list_long_df,sort=False)
df_long.reset_index(drop=True, inplace=True) 

df_short = pd.concat(list_short_df,sort=False)
df_short.reset_index(drop=True, inplace=True) 

df_mid = pd.concat(list_mid_df,sort=False)
df_mid.reset_index(drop=True, inplace=True) 


In [ ]:
'''
短时段在家的情况：totaltimes[key]<300
'''
zones_info_short=FinalAnalysis(df_short,"short",len(list_short_df)*5,number_cluster,
                                epsilon,row_num,column_num,map_img,res_path,unit,
                                max_x,min_x,max_y,min_y,groups,withTimeLine=1)

'''
长时段在家的情况：totaltimes[key]>600
'''
zones_info_long=FinalAnalysis(df_long,"long",len(list_long_df)*5,number_cluster,
                                epsilon,row_num,column_num,map_img,res_path,unit,
                                max_x,min_x,max_y,min_y,groups,withTimeLine=1)

print("===========> DONE <===========")

'''
中时段在家情况：totaltime[key]∈[300,600]
'''
zones_info_mid=FinalAnalysis(df_mid,"mid",len(list_mid_df)*5,number_cluster,
                                epsilon,row_num,column_num,map_img,res_path,unit,
                                max_x,min_x,max_y,min_y,groups,withTimeLine=1)

print("===========> DONE <===========")



In [ ]:
'''
data:df_long  data_allInOne  [Dataframe]
zone_info: zones_info_long   zones_info_forall  [Dataframe]
[zone分区编号  labels(list)所含网格编号  Area]
'''
def stay_time(data,zone_info,key):
    # key:分区名称-value:对应的分区字典[key:时间段-value:出现的次数]
    dict_periods={}
    # 高于6分钟的时间段占总时间段的比例  All:所有时间段数量  MTS: 高于6分钟时间段的数量
    MTS_info=pd.DataFrame(columns=['zone','All','MTS','MTSPercentage'])
    for idx,row in zone_info.iterrows(): # 每个分区
        labels_now=row['labels'] # 分区所含的所有网格编号
        periods=[]  # 所有时间段 (这一块儿是统计时长的？@段梦凡)
        
        count=0 # 计数 时间段长度
        for id,ro in data.iterrows():
            if ro['label'] in labels_now:
                count=count+1
            else:
                if count!=0:
                    periods.append(count)
                count=0
        set_periods=set(periods)
        dict={}
        for item in set_periods:
            dict.update({item:periods.count(item)}) # 某长度时间段出现的次数
        dict_periods.update({row['zone']:dict})
        
        pd1 = pd.DataFrame.from_dict(dict, orient='index')
        print(pd1)
        writer = pd.ExcelWriter(datadir+'tmp/'+key+'_'+str(idx)+'_periods.xlsx')
        # write dataframe to excel
        pd1.to_excel(writer)
        # save the excel
        writer.save()

        # 画柱状图
        plt.figure()
        p1=plt.bar(list(dict.keys()),list(dict.values()) ,0.4,color="green")
        plt.bar_label(p1, label_type='edge',fontsize=12)
        plt.xlabel('Time interval/min')
        plt.ylabel('Counts')
        plt.savefig(res_path+key+"_zone="+str(row['zone'])+"_TimeIntervalCounts"+".png",dpi=300)
        
        # 计算数字特征
        MoreThanSix=0
        MTSlength=0
        AllPeriods=0
        Alllength=0
        for i in dict.keys():
            if int(i)>=6:
                MoreThanSix=MoreThanSix+dict[i]
                MTSlength=MTSlength+int(i)*dict[i]
            AllPeriods=AllPeriods+dict[i]
            Alllength=Alllength+int(i)*dict[i]
        MTSPercentage=MoreThanSix/AllPeriods
        print("zone:"+str(row['zone'])+":MTS="+str(MTSPercentage))

        
        newdf = pd.DataFrame({'zone':[ row['zone']],
        'All':[AllPeriods],
        'MTS':[MoreThanSix],
        'MTSPercentage':[MTSPercentage]})
        MTS_info=MTS_info.append(newdf,ignore_index=True)

    # 画MTS信息图
    # 绘柱状图
    plt.cla() #清空之前画的
    fig,ax1 = plt.subplots()
    ax2 = ax1.twinx() 
    
    x=list(MTS_info['zone'])
    y1=list(MTS_info['All'])
    y2=list(MTS_info['MTS'])
    y3=list(MTS_info['MTSPercentage'])
    
    ax1.bar(x, height=y1, label='All Periods',width = 0.5, facecolor='lightskyblue',edgecolor='white')
    ax1.bar(x, height=y2, label='Periods > 6min',width = 0.5, facecolor='steelblue',edgecolor='white')

    ax1.set_xlabel("zone")
    ax1.legend()
    
    # 画折线图
    ax2.plot(x, y3, "coral", ms=8, marker='*',label="MTS Percentage")
    # 在折线图上显示具体数值, ha参数控制水平对齐方式, va控制垂直对齐方式
    for x1, yy in zip(x, y3):
        ax2.text(x1, yy+0.02, str(round(yy, 3)), ha='center', va='bottom', fontsize=10, rotation=0)
    ax2.legend(loc="upper left")
    plt.savefig(res_path+key+"_MTSinfo"+".png",dpi=300)

    return dict_periods,MTS_info


periods_info_all,MTS_info_all=stay_time(data_allInOne,zones_info_forall,"All")
periods_info_long,MTS_info_long=stay_time(df_long,zones_info_long,"long")
periods_info_short,MTS_info_short=stay_time(df_short,zones_info_short,"short")
periods_info_mid,MTS_info_mid=stay_time(df_mid,zones_info_mid,"mid")